# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv = "output_data/cities.csv"
city_data = pd.read_csv(city_csv)
city_data.describe()


,City ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,599.000000,599.000000,599.000000,599.000000,599.000000,599.00000,599.000000,5.990000e+02
mean,299.000000,19.864681,19.021243,51.345209,71.315526,52.00000,8.091369,1.613027e+09
std,173.060683,32.577710,88.861667,32.739050,21.018759,41.36844,6.396944,1.437105e+02
min,0.000000,-54.800000,-179.166700,-43.600000,12.000000,0.00000,0.250000,1.613026e+09
25%,149.500000,-7.324900,-57.655150,28.165000,60.500000,1.00000,3.440000,1.613026e+09
50%,299.000000,22.239400,25.133300,64.400000,76.000000,62.00000,6.600000,1.613027e+09
75%,448.500000,46.898250,91.906100,77.000000,87.000000,96.50000,10.415000,1.613027e+09
max,598.000000,78.218600,176.633300,98.600000,100.000000,100.00000,42.570000,1.613027e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

In [4]:
humidity_data = city_data[["Lat", "Lng","Humidity"]]
hm_locations = humidity_data[["Lat", "Lng"]].astype(float)
weights = humidity_data["Humidity"].astype(float)

In [5]:
heat_layer = gmaps.heatmap_layer(hm_locations, 
                                 weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3
                                )

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [6]:
ideal_conditions = city_data.loc[(city_data["Max Temp"] >= 70) & 
                                 (city_data["Max Temp"] < 80) & 
                                 (city_data["Wind Speed"] < 10) & 
                                 (city_data["Cloudiness"] == 0),:
                                ]
if len(ideal_conditions) > 10:
    ideal_conditions = ideal_conditions.head(10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(ideal_conditions)

hotel_df["Hotel Name"] = ""

hotels = []

for index, city in hotel_df.iterrows():
    
    hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = city['Lat']
    long = city['Lng']
    
    params = {
        "location": f'{lat},{long}',
        "keyword": 'hotel',
        "radius": 5000,
        "key": g_key
    }

    response = requests.get(hotel_url, params=params).json()
    try:
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append("")
    
hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!="",:]
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,hobart,-42.8794,147.3294,75.00,40,0,8.05,AU,1613026256,Hotel Grand Chancellor Hobart
20,20,road town,18.4167,-64.6167,75.99,89,0,1.01,VG,1613026578,Wyndham Tortola BVI Lambert Beach Resort
75,75,general roca,-39.0333,-67.5833,73.40,46,0,9.22,AR,1613026398,Estepa Hotel
82,82,lalmohan,22.3384,90.7419,78.37,37,0,4.12,BD,1613026587,Arafat Hotel
107,107,pochutla,15.7432,-96.4661,72.00,83,0,1.99,MX,1613026451,Hotel Posada San Jose
139,139,arraial do cabo,-22.9661,-42.0278,73.26,82,0,2.98,BR,1613026071,Orlanova Boutique Hotel
148,148,hermanus,-34.4187,19.2345,75.00,71,0,4.00,ZA,1613026106,The Whale Coast Hotel
256,256,caravelas,-17.7125,-39.2481,74.03,88,0,2.75,BR,1613026611,Mar Doce de Abrolhos


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))